In [55]:
# read html file and get table
import bs4
import re
import pandas as pd

file_path = 'test2.html'
fumen = pd.DataFrame()
read_file = open(file_path, 'r')
soup = bs4.BeautifulSoup(read_file, 'html.parser')

In [135]:
def parse(content):
    if content.name == 'img':
        if content.get('src') == '../t.gif':
            return('soflan')
        elif content.get('src') == '../b.gif' or content.get('src') == '../w.gif':
            # print(content.get('style'))
            # use regex to get the two numbers
            reg = re.compile(r'\d+')
            top, left = reg.findall(content.get('style'))
            
            return(top, left)
        elif content.get('src') == '../b2.gif' or content.get('src') == '../s.gif': # scrach
            return('scrach')
    elif content.name == 'span':
        print(content.text)
        return("bpm")

In [136]:
# get table in soup 
soup_table = soup.find_all('table')

In [137]:
for table in soup_table[1:]: # 第一张表是原表
    len(table.find_all('tr'))
    for tr in table.find_all('tr'):
        
        for notes in tr.find('div').contents:
            print(notes)
            print(parse(notes))
        mes = tr.find('th').text
        print(mes)


<img name="ttl1" onmousedown="dragOn('ttl1')" src="../w.gif" style="top:59px;left:65px"/>
('59', '65')
<img name="ttl2" onmousedown="dragOn('ttl2')" src="../b.gif" style="top:43px;left:107px"/>
('43', '107')
<img name="ttl3" onmousedown="dragOn('ttl3')" src="../b.gif" style="top:27px;left:51px"/>
('27', '51')
<img name="ttl4" onmousedown="dragOn('ttl4')" src="../w.gif" style="top:27px;left:93px"/>
('27', '93')
5
<img name="ttl5" onmousedown="dragOn('ttl5')" src="../s.gif" style="top:11px;left:0px"/>
scrach
4
3
<img height="2" src="../t.gif" style="top:126px" width="134"/>
soflan
<span style="z-index:5;font-size:10px;top:121px;font-weight:bold;width:20px;text-align:center;background:gray;left:140px;color:palegreen;">290</span>
290
bpm
2
<img name="ttl6" onmousedown="dragOn('ttl6')" src="../w.gif" style="top:123px;left:65px"/>
('123', '65')
<img name="ttl7" onmousedown="dragOn('ttl7')" src="../w.gif" style="top:115px;left:37px"/>
('115', '37')
<img name="ttl8" onmousedown="dragOn('ttl8')

In [60]:
def parse(content):
    if content.name == 'img':
        print(content.get('style'))
    elif content.name == 'span':
        print(content.text)

149

In [24]:
'top:123px;left:79px'
# reg: get top and left
# top: (\d+)px;left: (\d+)px
re.findall(r'top: (\d+)px;left: (\d+)px', s[1].get("style"))

'top:123px;left:79px'

In [70]:
notes.